In [ ]:
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import geopandas as gpd
import contextily as ctx
import urbanpy as up
from tqdm.notebook import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
gdf_celdas = gpd.read_parquet("outputs/celdas_02_06_2025.parquet")

In [ ]:
peru = up.download.nominatim_osm("Peru", 0, email="claudio@autodash.org")

In [ ]:
peru

In [ ]:
peru.plot()

In [ ]:
huancabamba = up.download.nominatim_osm(
    "Huancabamba, Peru", 0, email="claudio@autodash.org"
)

In [ ]:
huancabamba.crs

In [ ]:
huancabamba.plot()

In [ ]:
gdf_celdas.columns

In [ ]:
gdf_celdas["pop_2020"].describe()

In [ ]:
gdf_celdas[gdf_celdas["pop_2020"] == 0].shape

In [ ]:
gdf_celdas[gdf_celdas["pop_2020"] == 0]

In [ ]:
gdf_celdas["smod"].value_counts()

In [ ]:
gdf_celdas["smod"].value_counts(normalize=True) * 100

In [ ]:
gdf_celdas[gdf_celdas["smod"] != "rural"].shape

In [ ]:
gdf_celdas[(gdf_celdas["pop_2020"] > 1) & (gdf_celdas["smod"] != "rural")][
    "pop_2020"
].describe()

In [ ]:
celdas_huancabamba = gdf_celdas[gdf_celdas.intersects(huancabamba.geometry[0])]

In [ ]:
celdas_huancabamba.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

huancabamba.plot(ax=ax, facecolor="none", edgecolor="black")
celdas_huancabamba.plot(
    "pop_2020", cmap="viridis", legend=True, ax=ax, alpha=0.5, edgecolor="black"
)

In [ ]:
# Schools
schools, metadata = up.download.overpass(
    type_of_data="node",
    mask=huancabamba,
    query={
        "amenity": "school",
    },
)

In [ ]:
schools.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

huancabamba.plot(ax=ax, facecolor="none", edgecolor="black")
celdas_huancabamba.plot(
    "pop_2020", cmap="viridis", legend=True, ax=ax, alpha=0.5, edgecolor="black"
)
schools.plot(ax=ax, color="red", markersize=10)

# Add a basemap
# ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=huancabamba.crs)

In [ ]:
# start server
up.routing.start_osrm_server("peru", "south-america", "foot")

In [ ]:
huancabamba_access = up.accessibility.travel_times(
    celdas_huancabamba, schools, "schools"
)

In [ ]:
huancabamba_access.head()

In [ ]:
up.routing.stop_osrm_server("peru", "south-america", "foot")

In [ ]:
plotly.offline.init_notebook_mode()

In [ ]:
fig = up.plotting.choropleth_map(
    huancabamba_access,
    "pop_2020",
    title="Estimated Population - 2020",
    width=400,
    height=400,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))

fig.show()

In [ ]:
# Get ordered category labels
category_orders = (
    huancabamba_access["duration_to_nearest_schools_label"].unique().sort_values()
)
category_orders.categories

In [ ]:
fig = up.plotting.choropleth_map(
    huancabamba_access,
    color_column="duration_to_nearest_schools_label",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    category_orders={"duration_to_nearest_schools_label": category_orders},
    labels={"duration_to_nearest_schools_label": "Minutes"},
    title="Travel Time to Nearest School",
    width=400,
    height=400,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0))

# Remove the hexagon outlines to make the map clearer
fig.update_traces(marker_line_width=0)

fig.show()

In [ ]:
celdas_peru = gdf_celdas[gdf_celdas.intersects(peru.geometry[0])]

In [ ]:
celdas_peru.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

peru.plot(ax=ax, facecolor="none", edgecolor="black")
celdas_peru.plot(
    "pop_2020", cmap="viridis", legend=True, ax=ax, alpha=1, edgecolor="none"
)

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=peru.crs)

In [ ]:
# Schools
peru_schools, _ = up.download.overpass(
    type_of_data="node",
    mask=peru,
    query={
        "amenity": "school",
    },
)

In [ ]:
peru_schools.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

peru.plot(ax=ax, facecolor="none", edgecolor="black")

peru_schools.plot(ax=ax, color="red", markersize=0.001)

celdas_peru.plot(
    "pop_2020", cmap="viridis", legend=True, ax=ax, alpha=1, edgecolor="none"
)


# Add a basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=peru.crs)

In [ ]:
# start server
up.routing.start_osrm_server("peru", "south-america", "foot")

In [ ]:
peru_access = up.accessibility.travel_times(celdas_peru, peru_schools, "schools")

In [ ]:
peru_access.head()

In [ ]:
up.routing.stop_osrm_server("peru", "south-america", "foot")

In [ ]:
plotly.offline.init_notebook_mode()

In [ ]:
fig = up.plotting.choropleth_map(
    peru_access,
    "pop_2020",
    title="Estimated Population - 2020",
    opacity=0.5,
    width=800,
    height=800,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))

fig.show()

In [ ]:
# Get ordered category labels
category_orders = (
    huancabamba_access["duration_to_nearest_schools_label"].unique().sort_values()
)
category_orders.categories

In [ ]:
fig = up.plotting.choropleth_map(
    peru_access,
    color_column="duration_to_nearest_schools_label",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    category_orders={"duration_to_nearest_schools_label": category_orders},
    opacity=0.5,
    labels={"duration_to_nearest_schools_label": "Minutes"},
    title="Travel Time to Nearest School",
    width=800,
    height=800,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))

# Remove the hexagon outlines to make the map clearer
fig.update_traces(marker_line_width=0)

fig.show()